In [ ]:
import os
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats
from torch_geometric.data import InMemoryDataset, DataLoader
from torch_geometric import data as DATA
from lifelines.utils import concordance_index
from sklearn.metrics import mean_absolute_error
import torch
import pickle

import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

import sys, os
from random import shuffle

Much of this code is from/is adapted from https://github.com/thinng/GraphDTA
There paper is cited as
Nguyen, T., Le, H., Quinn, T. P., Nguyen, T., Le, T. D., & Venkatesh, S. (2021). GraphDTA:
Predicting drug–target binding affinity with graph neural networks. Bioinformatics, 37(8), 1140-1147.

In [10]:
class TestbedDataset(InMemoryDataset):
    def __init__(self, root='/tmpns', dataset='',
                 xd=None, y=None, transform=None,
                 pre_transform=None,smile_graph=None, tu = False):

        #root is required for save preprocessed data, default is '/tmp'
        super(TestbedDataset, self).__init__(root, transform, pre_transform)
        self.dataset = dataset
        if (os.path.isfile(self.processed_paths[0]) and tu):
            print('Pre-processed data found: {}, loading ...'.format(self.processed_paths[0]))
            self.data, self.slices = torch.load(self.processed_paths[0])
        else:
            print('Pre-processed data {} not found, doing pre-processing...'.format(self.processed_paths[0]))
            self.process(xd, y,smile_graph)
            self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return [self.dataset + '.pt']

    def _process(self):
        if not os.path.exists(self.processed_dir):
            os.makedirs(self.processed_dir)


    def process(self, xd, y,smile_graph):
        assert (len(xd) == len(y)), "The three lists must be the same length!"
        data_list = []
        data_len = len(xd)
        for i in range(data_len):
            print('Converting SMILES to graph: {}/{}'.format(i+1, data_len))
            smiles = xd[i]
            labels = y[i]
            # convert SMILES to molecular representation using rdkit
            c_size, features, edge_index = smile_graph[smiles]
            # make the graph ready for PyTorch Geometrics GCN algorithms:
            GCNData = DATA.Data(x=torch.Tensor(features),
                                edge_index=torch.LongTensor(edge_index).transpose(1, 0),
                                y=torch.FloatTensor([labels]))
            GCNData.__setitem__('c_size', torch.LongTensor([c_size]))
            # append graph, label and target sequence to data list
            data_list.append(GCNData)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        print('Graph construction done. Saving to file.')
        data, slices = self.collate(data_list)
        # save preprocessed data:
        torch.save((data, slices), self.processed_paths[0])

def rmse(y,f):
    rmse = sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse

def mae(y,f):
    return mean_absolute_error(y,f)

def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
def spearman(y,f):
    rs = stats.spearmanr(y, f)[0]
    return rs

def ci(y,f):
    return concordance_index(y,f)


In [12]:
def atom_features(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na','Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb','Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H','Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr','Cr', 'Pt', 'Hg', 'Pb', 'Unknown']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6,7,8,9,10]) +
                    [atom.GetIsAromatic()])

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def smile_to_graph(smile):
    mol = Chem.MolFromSmiles(smile)

    c_size = mol.GetNumAtoms()

    features = []
    for atom in mol.GetAtoms():
        feature = atom_features(atom)
        features.append( feature / sum(feature) )

    edges = []
    for bond in mol.GetBonds():
        edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
    g = nx.Graph(edges).to_directed()
    edge_index = []
    for e1, e2 in g.edges:
        edge_index.append([e1, e2])

    return c_size, features, edge_index

In [13]:
with open("smile_graph.pkl", 'rb') as handle:
  smile_graph = pickle.load(handle)

In [14]:
class GINConvNet(torch.nn.Module):
    def __init__(self, n_output=1,num_features_xd=78, num_features_xt=25,
                 n_filters=32, embed_dim=128, output_dim=128, dropout=0.2):

        super(GINConvNet, self).__init__()

        dim = 32
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.n_output = n_output

        nn1 = Sequential(Linear(num_features_xd, dim), ReLU(), Linear(dim, dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(dim)

        nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = torch.nn.BatchNorm1d(dim)

        nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv3 = GINConv(nn3)
        self.bn3 = torch.nn.BatchNorm1d(dim)

        nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv4 = GINConv(nn4)
        self.bn4 = torch.nn.BatchNorm1d(dim)

        nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv5 = GINConv(nn5)
        self.bn5 = torch.nn.BatchNorm1d(dim)

        self.fc1_xd = Linear(dim, int((dim*2)/3))


        self.fc2_xd = Linear(int((dim*2)/3), int((dim*4)/9))

        self.out = nn.Linear(int((dim*4)/9), self.n_output)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = F.relu(self.conv5(x, edge_index))
        x = self.bn5(x)
        x = global_add_pool(x, batch)
        x = F.relu(self.fc2_xd(F.relu(self.fc1_xd(x))))
        x = F.dropout(x, p=0.2, training=self.training)

        out = self.out(x)
        return out

In [17]:
def train(model, device, train_loader, optimizer, epoch):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * len(data.x),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data.y.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(),total_preds.numpy().flatten()



In [19]:
df = pd.read_csv('test.csv')
test_drugs, test_Y = list(df['SMILES']),list(df['3CLPro_pocket1'])
test_drugs, test_Y = np.asarray(test_drugs), np.asarray(test_Y)

In [ ]:
targs = ['3CLPro_pocket1', 'ADRP-ADPR_pocket1', 'ADRP-ADPR_pocket5',
       'ADRP_pocket1', 'ADRP_pocket12', 'ADRP_pocket13', 'COV_pocket1',
       'COV_pocket2', 'COV_pocket8', 'COV_pocket10', 'NSP9_pocket2',
       'NSP9_pocket7', 'NSP15_pocket1', 'ORF7A_pocket2',
       'PLPro_chainA_pocket3', 'PLPro_chainA_pocket23', 'PLPro_pocket6',
       'PLPro_pocket50']
for i in targs:
  df = pd.read_csv('train.csv')
  train_drugs, train_Y = list(df['SMILES']),list(df[i])
  train_drugs, train_Y = np.asarray(train_drugs), np.asarray(train_Y)
  print('test: ')
  test_data = TestbedDataset(dataset='test.csv', xd=test_drugs, y=test_Y,smile_graph=smile_graph, tu = True)
  print('train: ')
  train_data = TestbedDataset(dataset='train.csv', xd=train_drugs, y=train_Y,smile_graph=smile_graph, tu = False)

  TRAIN_BATCH_SIZE = 512
  TEST_BATCH_SIZE = 512
  LR = 0.0005
  LOG_INTERVAL = 20
  NUM_EPOCHS = 1000

  train_size = int(0.8 * len(train_data))
  valid_size = len(train_data) - train_size
  train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])

  train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
  valid_loader = DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
  test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

  device = torch.device("cuda:{}".format(0)) if torch.cuda.is_available() else torch.device("cpu")
  model = GINConvNet().to(device)
  loss_fn = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=LR)


  saved_p = {}
  mse_d = {}
  mae_d = {}
  ci_d = {}
  best_mse = 1000
  best_mae = 1000
  best_ci = 0
  best_epoch = -1

  for epoch in range(NUM_EPOCHS):
    train(model, device, train_loader, optimizer, epoch+1)
    print('predicting for valid data')
    G,P = predicting(model, device, valid_loader)
    ret = [mse(G,P),mae(G,P),ci(G,P)]
    mse_d[epoch] = {ret[0]}
    mae_d[epoch] = {ret[1]}
    ci_d[epoch] = {ret[2]}
    val = ret[1]
    if val<best_mae:
      best_mae = val
      best_mse = ret[0]
      best_ci = ret[-1]
      best_epoch = epoch+1
      model_file_name = '1000yikes' + i + str(epoch) + '.model'
      torch.save(model.state_dict(), model_file_name)
      print('predicting for test data')
      G,P = predicting(model, device, test_loader)
      saved_p[epoch] = P
      print('rmse improved at epoch ', best_epoch, '; best_mae,best_mse,best_ci:', best_mae,best_mse,best_ci) #,model_st)
    else:
      print('no improvement since epoch ', best_epoch, '; best_mae,best_mse,best_ci:', best_mae,best_mse,best_ci) #,model_st)

## For Binary_Loss_Incorporated_Convolutional_Graph_Isomorphism_Network the following modifications were made to the code

In [ ]:
import pandas as pd
import plotly.express as px

#ReArranged train set so that the values further (at least .04) away from -6.93 are at the bottom of the set and thus only used in the validation set (this is to reduce throwing off loss due to small perterbations in the predicted value)
df = pd.read_csv('ntrain.csv')

fig = px.histogram(df, x='3CLPro_pocket1', nbins=7000, opacity=0.7)
fig.update_layout(title='Distribution of 3CLPro_pocket1', xaxis_title='Values', yaxis_title='Frequency')
fig.show()

In [ ]:
#Train function was modified to include variable bin_labels which was just the set of batch split binary encoded vectors for the train set in lue of these vectors being in the test bed
def train(model, device, train_loader, optimizer, epoch, bin_labels):
    print('Training on {} samples...'.format(len(train_loader.dataset)))
    model.train()
    c = 0
    sig = nn.Sigmoid()
    for batch_idx, data in enumerate(train_loader):
        binar = np.asarray(bin_labels[c])
        c = c + 1
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)


        #The following postion of the train function was modified to include binary loss into our loss function
        ################################################################################################
        threshold = -6.93

        binary_tensor = torch.zeros_like(output)
        binary_tensor[output > threshold] = 1.0
        a = binary_tensor
        b = torch.tensor(binar, dtype=torch.float32)
        b = b.view(a.shape)
        b = b.to(device)
        lb = torch.abs(torch.sum(torch.sub(a, b)))
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device)) + lb

        ################################################################################################


        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                                                                           batch_idx * len(data.x),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.item()))

In [ ]:
#The following code in the train-validation loop was changed so that the order was no longer shuffled, and the split ensured that the train set did not have affinity values close to defined threshold of -6.93

train_size = int(0.8 * len(train_data))
valid_size = len(train_data) - train_size

valid_data = train_data[train_size:train_size + valid_size]
train_data = train_data[:train_size]

bat = list(df['encoded_array'])
input_list = bat[:train_size]
TRAIN_BATCH_SIZE = 512
bin_labels = [input_list[i:i+TRAIN_BATCH_SIZE] for i in range(0, len(input_list), TRAIN_BATCH_SIZE)]


train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=False)
valid_loader = DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

device = torch.device("cuda:{}".format(0)) if torch.cuda.is_available() else torch.device("cpu")
model = GINConvNet().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


saved_p = {}
mse_d = {}
mae_d = {}
ci_d = {}
best_mse = 1000
best_mae = 1000
best_ci = 0
best_epoch = -1

for epoch in range(NUM_EPOCHS):
  train(model, device, train_loader, optimizer, epoch+1, bin_labels)